In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [17]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
final_dataset=ratings.pivot(index="movieId", columns="userId", values="rating")
final_dataset.fillna(0,inplace=True)
no_of_users_who_rated_movie = ratings.groupby('movieId')['userId'].agg('count')
final_dataset=final_dataset.loc[no_of_users_who_rated_movie[no_of_users_who_rated_movie>10].index,:]
movies_rated_by_users = ratings.groupby('userId')['movieId'].agg('count')
final_dataset = final_dataset.loc[:,movies_rated_by_users[movies_rated_by_users>50].index]
csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)

In [18]:
def movie_rec(movie_name):
    movie_list = movies[movies['title'].str.contains(movie_name)]
    movie_index = movie_list.iloc[0]['movieId']
    knn=NearestNeighbors(metric='cosine',n_neighbors=20)
    knn.fit(csr_data)
    similarities,indeces=knn.kneighbors(csr_data[final_dataset[final_dataset['movieId'] == movie_index].index[0]],n_neighbors=11)
    rec = sorted(zip(indeces.squeeze().tolist(),similarities.squeeze().tolist()),key=lambda x:x[1])[:0:-1]
    l=[]
    for val in rec:
        movie_Id = final_dataset.iloc[val[0]]['movieId']
        l.append(movies[movies['movieId']==movie_Id]['title'].values[0])
        print(movies[movies['movieId']==movie_Id]['title'].values[0])
    return l

In [14]:
def movie(name):
    get_name = name.split(' ')
    correct_name=''
    for index in range(len(get_name)):
        if index==0 and get_name[index].isalpha():
            get_name[index]=get_name[index].capitalize()
            correct_name+=get_name[index]
        elif get_name[index].isalpha():
            get_name[index]=get_name[index].capitalize()
            correct_name+=' '+get_name[index]
        else:
            correct_name+=' '+get_name[index]
        print(correct_name)
    return movie_rec(correct_name)

In [15]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [28]:
movie_name = 'The avengers'

In [33]:
# Preprocess the titles
movies['clean_title'] = movies['title'].str.lower().str.split('(').str[0].str.strip()

In [34]:
movie_list = movies[movies['clean_title'].str.contains(movie_name.lower(), na=False)]

In [38]:
from fuzzywuzzy import process

def fuzzy_search(movie_name, choices, limit=5):
    return process.extract(movie_name, choices, limit=limit)

# Use fuzzy matching to find top 5 matches
matches = fuzzy_search(movie_name.lower(), movies['clean_title'])
print(matches)  # Returns a list of tuples (matched_title, score, index)

/Applications/websites/movie_recom/movie_recomend/env/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[('avengers, the', 95, 1611), ('avengers, the', 95, 7693), ('father of the bride part ii', 86, 4), ('american president, the', 86, 10), ('city of lost children, the', 86, 28)]


In [ ]:
from fuzzywuzzy import process

def movie_rec(movie_name):
    # Preprocess titles
    movies['clean_title'] = movies['title'].str.lower().str.split('(').str[0].str.strip()
    # Fuzzy matching
    matches = process.extract(movie_name.lower(), movies['clean_title'], limit=5)
    if not matches:
        print(f"No matches found for '{movie_name}'")
        return
    # Pick the best match above a confidence threshold
    best_match = matches[0]
    if best_match[1] >= 75:  # Confidence score
        movie_index = movies.iloc[best_match[2]]['movieId']
        print(f"Match found: {best_match[0]} (MovieId: {movie_index})")
        # Continue with KNN or other logic...
        # movie_index = movie_list.iloc[0]['movieId']
        knn=NearestNeighbors(metric='cosine',n_neighbors=20)
        knn.fit(csr_data)
        similarities,indeces=knn.kneighbors(csr_data[final_dataset[final_dataset['movieId'] == movie_index].index[0]],n_neighbors=11)
        rec = sorted(zip(indeces.squeeze().tolist(),similarities.squeeze().tolist()),key=lambda x:x[1])[:0:-1]
        l=[]
        for val in rec:
            movie_Id = final_dataset.iloc[val[0]]['movieId']
            l.append(movies[movies['movieId']==movie_Id]['title'].values[0])
            print(movies[movies['movieId']==movie_Id]['title'].values[0])
        return l
    else:
        print(f"No good match found for '{movie_name}'")

In [43]:
movie_rec('The avengers')

Match found: avengers, the (MovieId: 2153)
Matilda (1996)
Haunting, The (1999)
Mercury Rising (1998)
Speed 2: Cruise Control (1997)
Reindeer Games (2000)
Snake Eyes (1998)
William Shakespeare's A Midsummer Night's Dream (1999)
Anaconda (1997)
Edge, The (1997)
Sphere (1998)


['Matilda (1996)',
 'Haunting, The (1999)',
 'Mercury Rising (1998)',
 'Speed 2: Cruise Control (1997)',
 'Reindeer Games (2000)',
 'Snake Eyes (1998)',
 "William Shakespeare's A Midsummer Night's Dream (1999)",
 'Anaconda (1997)',
 'Edge, The (1997)',
 'Sphere (1998)']